## Cleaning data and creating dummy variables 
	Create variable for length of description
	Create Cats_Allowed and Dogs_Allowed dummy variables
	Amenities Dummy Variables
	Cityname (if less than 50 records per city set to 'Other')
	Droped rows with nulls for ['price','zipcode','perc_sub25k','bedrooms','bathrooms','POPULATION','state']
		df after (87063, 56)

### Loading libraries

In [2]:
import pandas as pd
import numpy as np

### Loading Data

In [2]:
df = pd.read_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/merged_data.csv', 
                 encoding='utf-8',
                 index_col=0)

df.columns
df

C:\Users\sega9\AppData\Local\Temp\ipykernel_10896\919938043.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/merged_data.csv',


,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,POP_SQMI,SQMI,Shape_Length,Shape_Area,perc_sub25k,perc_25-50k,perc_50-75k,perc_75-100k,perc_100-200k,perc_abv200k
0,5668640009,housing/rent/apartment,One BR 507 & 509 Esplanade,"This unit is located at 507 & 509 Esplanade, R...",NaN,1.0,1.0,USD,No,Thumbnail,...,7463.89,6.12,0.164548,0.001543,0.241601,0.171652,0.133916,0.106765,0.224114,0.121951
1,5668639818,housing/rent/apartment,Three BR 146 Lochview Drive,"This unit is located at 146 Lochview Drive, Ne...",NaN,1.5,3.0,USD,No,Thumbnail,...,2908.02,10.48,0.265637,0.002751,0.315178,0.269426,0.147422,0.085694,0.139434,0.042847
2,5668639686,housing/rent/apartment,Three BR 3101 Morningside Drive,This unit is located at 3101 Morningside Drive...,NaN,2.0,3.0,USD,No,Thumbnail,...,3288.26,11.33,0.271071,0.002928,0.276393,0.224403,0.147480,0.084881,0.151194,0.115650
3,5668639659,housing/rent/apartment,Two BR 209 Aegean Way,"This unit is located at 209 Aegean Way, Vacavi...",NaN,1.0,2.0,USD,No,Thumbnail,...,403.04,91.00,0.798495,0.024312,0.234981,0.202020,0.133440,0.104200,0.226475,0.098884
4,5668639374,housing/rent/apartment,One BR 4805 Marquette NE,"This unit is located at 4805 Marquette NE, Alb...",NaN,1.0,1.0,USD,No,Thumbnail,...,2088.14,12.73,0.457831,0.003256,0.397260,0.253425,0.122432,0.069349,0.111301,0.046233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99462,5121219946,housing/rent/apartment,Houston - superb Apartment nearby fine dining,"Redefining urban living, in southeast Houston,...","Gym,Parking,Patio/Deck,Playground,Storage,Wood...",1.0,1.0,USD,No,Yes,...,2502.54,14.16,0.334310,0.003417,0.437771,0.313932,0.136223,0.057585,0.050155,0.004334
99463,5121219696,housing/rent/apartment,The Best of the Best in the City of Jacksonvil...,Courtney Manor Apartments offer the best of ev...,"AC,Cable or Satellite,Clubhouse,Dishwasher,Gym...",2.0,2.0,USD,No,Yes,...,2448.10,25.26,0.454914,0.006131,0.382037,0.332249,0.143508,0.070290,0.066059,0.005857
99464,5121219420,housing/rent/apartment,A great & large One BR apartment. Pet OK!,"Fully remodeled, new floor, kitchen cabinet, s...","Garbage Disposal,Refrigerator",1.0,1.0,USD,No,Yes,...,12219.16,5.69,0.212834,0.001418,0.410072,0.339693,0.132937,0.058805,0.051924,0.006569
99465,5121218935,housing/rent/apartment,"The Crest offers studio, 1, 2 & Three BR homes...","Amenities include a fitness facilities, swimmi...","Gym,Pool",1.0,1.0,USD,No,Yes,...,1088.35,64.91,0.957747,0.016681,0.236364,0.167643,0.126656,0.097072,0.246841,0.125424


### Creating Variables

##### Description

In [ ]:
# Create variable for length of description
# ------------------------------------------------------------------------------------
df['description_len'] = df['body'].apply(len)

##### Pets

In [ ]:
# Create Cats_Allowed and Dogs_Allowed dummy variables
# ------------------------------------------------------------------------------------
df['cats_allowed'] = df['pets_allowed'].apply(lambda x: 1 if isinstance(x, str) and 'Cats' in x else 0)
df['dogs_allowed'] = df['pets_allowed'].apply(lambda x: 1 if isinstance(x, str) and 'Dogs' in x else 0)

##### Ameneties

In [ ]:
# Amenities Dummy Variables
# ------------------------------------------------------------------------------------
amen = pd.Series(df['amenities']).astype(str)
# Split by "," and collapse
amen = amen.str.split(",").explode()
# Keep unique values
amen_raw = pd.Series(amen).unique()
# Replace spaces for use in colnames
amen_name = pd.Series(amen_raw).str.replace(" ", "_").str.replace("/", "_")
amen_name = "has_" + amen_name
# Save colnames to add to the final DataFrame
b4 = df.columns.tolist()
# For each amenity, create new dummy variable
for i in amen_raw:
    new_col = pd.Series(df['amenities']).fillna('').str.contains(i).astype(int)
    df[i] = new_col
# Rename added columns
df.columns = b4 + amen_name.tolist()

##### Cities

In [ ]:
# Cityname
# ------------------------------------------------------------------------------------
# Count occurrences of each city
city_counts = df['cityname'].value_counts()
# Identify cities with less than 50 records
less_than_50 = city_counts[city_counts < 50].index

# Set values with less than 50 records to "Other"
df.loc[df['cityname'].isin(less_than_50), 'cityname'] = 'Other'

##### Price Type

In [ ]:
# Price "Monthly Weekly Monthly|Weekly"
# ------------------------------------------------------------------------------------
# Display unique values in 'price_type'
df['price_type'].unique()

# Count rows where 'price_type' is 'Monthly|Weekly'
print(df[df['price_type'] == 'Monthly|Weekly'].shape[0])
# Count rows where 'price_type' is 'Weekly'
print(df[df['price_type'] == 'Weekly'].shape[0])

# only 4 records with non 'Montly' -> Drop rows where 'price_type' is 'Monthly|Weekly' or 'Weekly'
df = df[~df['price_type'].isin(['Monthly|Weekly', 'Weekly'])]

### Drop Redundant/Unused Columns

In [ ]:
# Drop redundant Cols
# ------------------------------------------------------------------------------------
# List of columns to drop
columns_to_drop = ['category', 'amenities', 'price_type', 'pets_allowed', 'time', 
                   'currency','price_display','index_right','PO_NAME','Shape_Length',
                   'Shape_Area','has_nan','STATE']
# Drop the specified columns
df = df.drop(columns=columns_to_drop)

df.shape

#### Drop Empty Values

In [ ]:
# Drop empty values
df = df[df['price'].notna()]
df = df[df['zipcode'].notna()]
df = df[df['perc_sub25k'].notna()]
df = df[df['bedrooms'].notna()]
df = df[df['bathrooms'].notna()]
df = df[df['POPULATION'].notna()]
df = df[df['state'].notna()]

#### Save Dataset

In [ ]:
# Save to CSV
df.to_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/feature_engineered_data.csv', 
          encoding='utf-8',
          index= False)